In [ ]:
import requests
import datetime
import json
import pandas as pd

In [ ]:
# get token

env_file = open(".env", mode="r")

token = env_file.read().split("=")[1]

In [ ]:
headers = {
    'Authorization': f"token {token}",
    'accept': "application/vnd.github.v3+json",
}
base_url = "https://api.github.com"
current_year = datetime.datetime.now().year
owner = "idec-teams"

In [ ]:
gh_session = requests.Session()
gh_session.headers.update(headers)

In [ ]:
# get all repositories under current year
get_repos_url = f"{base_url}/orgs/idec-teams/repos"
get_repos_response = gh_session.get(get_repos_url)
print (f"status code: {get_repos_response.status_code}")
response_json = get_repos_response.json()
current_year_repos = [response_item['name'] for response_item in response_json if f"{current_year}_" in response_item['name']]
current_year_repos = [repo for repo in current_year_repos if "trevor" not in repo]

print(f"{current_year} repositories:")

for repo in current_year_repos:
    print(f"\t{repo}")

In [ ]:
# For checking user permissions
# Re run this cell after wiki-freeze to determine if wiki-freeze change was successful

get_user_permission_responses_output = list()

for repo in current_year_repos:
    get_repo_collaborators = f"{base_url}/repos/{owner}/{repo}/collaborators"
    get_repo_collaborators_response = gh_session.get(get_repo_collaborators)

    response_json = get_repo_collaborators_response.json()
    collaborators = [response_item['login'] for response_item in response_json if response_item['permissions']['admin']==False]
    collaborators = [collaborator for collaborator in collaborators if collaborator != "idechq-bot"]

    for username in collaborators:
        get_collaborators_permission_url = f"{base_url}/repos/{owner}/{repo}/collaborators/{username}/permission"
        get_user_permission_response = gh_session.get(get_collaborators_permission_url)
        get_user_permission_responses_output.append(
            {"repo": repo,
            "username": username,
            "response": get_user_permission_response.status_code,
            "permission": get_user_permission_response.json()['permission']
            }
        )

pd.DataFrame(get_user_permission_responses_output)

In [ ]:
# Wiki freeze command

update_user_responses_output = list()

for repo in current_year_repos:
    get_repo_collaborators = f"{base_url}/repos/{owner}/{repo}/collaborators"
    get_repo_collaborators_response = gh_session.get(get_repo_collaborators)

    response_json = get_repo_collaborators_response.json()
    collaborators = [response_item['login'] for response_item in response_json if response_item['permissions']['admin']==False]
    collaborators = [collaborator for collaborator in collaborators if collaborator != "idechq-bot"]

    for username in collaborators:
        update_collaborator_url = f"{base_url}/repos/{owner}/{repo}/collaborators/{username}"
        update_user_response = gh_session.put(update_collaborator_url,
                                data=json.dumps({"permission": "pull"})
                                )
        update_user_responses_output.append(
            {"repo": repo,
            "username": username,
            "response": update_user_response.status_code,
            }
        )

pd.DataFrame(update_user_responses_output)